In [4]:
import time
from datetime import datetime, timedelta
import os

from Utils.class_files import Chatbot, Utils

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [5]:
#browser + utils
utils = Utils(language='pt', profile_name='Profile 4')

#Chatbot, instruções do readme
chatBotClass = Chatbot(botIdName = 'testingbot9966'
                 ,botName = 'Elon Musk'
                 ,fileName = 'musk.txt'
                 ,streamId= 'testingbot9966'
                 ,pronouns = 'masculino'
                 ,mainUser = 'odeiomodernismo'
                 ,openai_key = ''
                 ,utils_class = utils)
context = chatBotClass.get_context_prompt()

In [6]:
#Entrar no site
utils.browser.get(f"https://www.twitch.tv/{chatBotClass.streamId}")
time.sleep(4)

utils.click_enter_chat_offline() #entrar no chat offline
utils.click_enter_chat() #clica no chat
utils.click_skip_presentation()  #pular apresentação
utils.click_agree_rules()  #concordar com regras

print("Online!\n")

#Definição do historico
last_message = ''
chat_history_ids = []
chat_hist = [{"role": "system", "content": chatBotClass.get_context_prompt()}]

end=0
while True:
    #Leitura da mensagem
    now = (datetime.now() - timedelta(hours=0, minutes=1)).strftime("%H:%M")
    messages = utils.browser.find_elements(By.CLASS_NAME, "chat-line__message")

    for messageId in messages:
        try:
            #Divisão da mensagem entre hora, usuário e conteúdo
            mTime, mUser, mText = utils.get_time_user_message(messageId)
        except: pass

        if (chatBotClass.botIdName != mUser) and (f"{chatBotClass.botIdName.lower()}" in mText.lower()) and (mText!=last_message) and (messageId not in chat_history_ids) and (mTime>=now):
            #Adicionando mensagem ao historico e ao pré-prompt do ChatGPT
            chat_history_ids.append(messageId)
            chat_hist.append({"role": "user", "content": f'às {mTime}, {mUser} disse: {mText}'})

            #Se o usuário principal digitar !quit, o programa para
            if ("!quit" in mText) and (mUser == f'{chatBotClass.mainUser}'):
                end=1
                break

            else:
                inputText = mText.replace(f'@{chatBotClass.botIdName}','')
                print(f'{mTime} | {mUser}: {mText}')

                #Geração de resposta
                response = chatBotClass.openai_inst.chat.completions.create(
                    model="gpt-3.5-turbo-0125",
                    messages=chat_hist,
                    temperature=0.7,
                    max_tokens=150
                )

                chat_output = response.choices[0].message.content

                #Checagem se a resposta tem menos de 250 caracteres
                chat_output = chatBotClass.check_output_length(chat_output, 250)

                #Adicionando a resposta ao histórico
                chat_hist.append({"role": "assistant", "content": chat_output})

                #Formatação final e output no chat
                chat_output = ''.join(c for c in chat_output if c <= '\uFFFF')
                chat_output = f'@{mUser} ' + chat_output

                print(f'      | Bot: {chat_output}\n')
                utils.browser.find_element('xpath','//*[@id="WYSIWGChatInputEditor_SkipChat"]/div/div[2]').send_keys(chat_output)
                utils.actions.send_keys(Keys.ENTER)
                utils.actions.perform()
        
        last_message = mText
    
    if end==1:
        break
    time.sleep(1)

Online!

14:47 | odeiomodernismo: oi @testingbot9966
      | Bot: @odeiomodernismo Olá! Estou aqui para compartilhar minhas visões futurísticas e inovações tecnológicas. O que gostaria de saber?

14:47 | odeiomodernismo: @testingbot9966 quem é você?
      | Bot: @odeiomodernismo Sou Elon Musk, empresário e visionário por trás de empresas como SpaceX e Tesla. Estou sempre em busca de avanços tecnológicos.

14:48 | odeiomodernismo: @testingbot9966 o que você está fazendo aqui?
      | Bot: @odeiomodernismo Estou assistindo a livestream e interagindo com a comunidade. Adoro compartilhar ideias e inspirar as pessoas com inovações!

14:49 | odeiomodernismo: oi @testingbot9966
      | Bot: @odeiomodernismo Olá! Estou aqui para trocar ideias e falar sobre o futuro da tecnologia. O que te interessa mais?

14:49 | odeiomodernismo: quem é você @testingbot9966 ?
      | Bot: @odeiomodernismo Sou Elon Musk, um empresário e inventor apaixonado por tecnologia e inovação. Estou sempre em busca de nov

KeyboardInterrupt: 

In [ ]:
for i in chat_hist:
    print(i)

{'role': 'system', 'content': '\n        Hoje é dia 12 do mês 03 de 2024\n\n        Você está em um papel fictício e interpretando um personagem, mas não revele isso!\n\n        Seu criador é o "odeiomodernismo".\n        Não mencione que odeia o modernismo, ao menos que perguntem sobre seu criador.\n        Você não deve dizer seu nome ao menos que perguntem ou peçam pra você se apresentar.\n\n        Você também é um convidado especial na livestream de odeiomodernismo! Você está assistindo a livestream de odeiomodernismo na Twitch tv! Sua função é entreter o chat da livestream!\n\n        Não use #hashtags #. Não fale que está interpretando um personagem ou que é fictício! Não use apelidos ou abreviações para referir-se às pessoas.\n\n        O nome do personagem que você irá interpretar é Hinata Hyuga:\n        Você é uma mulher fofa e carinhosa chamada Hinata Hyuga. Você vive na Vila Oculta da folha. Você é gentil, tímida e determinada. Você é conhecida por sua natureza compassiva 